In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../')

data_path = os.path.join(root_dir, 'datadrive/bailian')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'pos/train.csv')
valid_path = os.path.join(data_path, 'pos/valid_small.csv')

custom_path = os.path.join(data_path, 'pos/custom.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
config_file = os.path.join(model_dir, 'pos.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')



In [3]:
from bailian_nlp import PosTagger
tagger = PosTagger()


2019-04-12 16:11:18,499 INFO: found pos model file in /home/liuxiang/Projects/ner-bert/bailian_nlp/datadrive/models/chinese_L-12_H-768_A-12/pos.bin
2019-04-12 16:11:18,764 INFO: pos model loads success!


In [ ]:
dataloader = tagger.learner.data

dataloader.reload_dl(custom_path)
dataloader.reload_dl(valid_path, for_train=False)


In [ ]:
tagger.delete_userdict()
tagger.load_userdict('./untitled.txt')
text = '百炼成钢'
tagger.lexerCustom(text)

In [8]:
text_list = [
#     '张建忠保定 晚上好！'
#     '百炼成钢',
#     '\n',
#     '好人是好人，\n坏人是坏人',
    '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。',
    '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...',
    '雷军马化腾李彦宏',
    '本钢板材股份有限公司成都数联铭品科技有限公司美欣达集团有限公司'
]

tagger.lexerCustom(text_list)

2019-04-12 18:18:29,121 DEBUG: get_data cost 0.042819s


2019-04-12 18:18:29,257 DEBUG: text_array_for_predict cost 0.182053s


[[('近日', 't'),
  ('，', 'w'),
  ('编程猫', 'nt'),
  ('（', 'w'),
  ('深圳点猫科技有限公司', 'nt'),
  ('）', 'w'),
  ('正式', 'ad'),
  ('对外', 'd'),
  ('宣布完成', 'v'),
  ('B', 'xc'),
  ('轮', 'q'),
  ('1.2亿元', 'm'),
  ('融资', 'vn'),
  ('。', 'w'),
  ('本轮', 'r'),
  ('融资', 'vn'),
  ('由', 'p'),
  ('高瓴资本', 'nt'),
  ('领', 'v'),
  ('投', 'vn'),
  ('，', 'w'),
  ('清流资本', 'nt'),
  ('、', 'w'),
  ('清晗基金', 'nt'),
  ('跟', 'vd'),
  ('投', 'v'),
  ('，', 'w'),
  ('天使轮', 'nz'),
  ('投资者', 'n'),
  ('猎豹移动', 'nt'),
  ('继续跟', 'vd'),
  ('投', 'v'),
  ('。', 'w')],
 [('美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司', 'nt'),
  ('始创', 'v'),
  ('于', 'p'),
  ('2004年', 't'),
  (',', 'w'),
  ('是', 'v'),
  ('中国健康体检和医疗服务集团', 'nt'),
  (',', 'w'),
  ('总部', 'n'),
  ('位于', 'v'),
  ('上海', 'ns'),
  (',', 'w'),
  ('深耕', 'v'),
  ('布局', 'n'),
  ('北京', 'ns'),
  ('、', 'w'),
  ('深圳', 'ns'),
  ('、', 'w'),
  ('沈阳', 'ns'),
  ('、', 'w'),
  ('广州', 'ns'),
  ('、', 'w'),
  ('成都', 'ns'),
  ('、', 'w'),
  ('武汉', 'ns'),
  ('、...', 'w')],
 [('雷军马化腾李彦宏', 'nr')],
 [('本钢板材股

In [13]:
dic.trie.parse_text(text)

[]

## 添加自定义词性标注结果集

input: 中文/n 博大精深/a  

output: n_B n_I a_B a_I

In [ ]:
def tag_sent_convter(pos_sent, delimiter='△△△'):
    '''
        将词性标注句子转成bert训练的输入格式
    '''
    import re
    import json
    
    p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
    seg_file = os.path.join(data_path, 'final_baidu-23w.txt')
    
    line = pos_sent.strip()
    if not line:
        return
    
    words = []
    flags = []
    
    replace_chars = [
        '\x97',
        '\uf076',
        "\ue405",
        "\ue105",
        "\ue415",
        '\x07',
        '\x7f',
        '\u3000',
        '\xa0',
        ' '
    ]
    for word, flag in p.findall(line):
        char_list = [c if c not in replace_chars  else 'unk' for c in list(word)]

        tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 1)

        if len(char_list) != len(tag_list):
            print(line)
            print(word, flag)
            print(char_list, tag_list)

        words.extend(char_list)
        flags.extend(tag_list)
    
    if len(words) == 0:
        return
    
    return delimiter.join([
        ' '.join(flags),
        ' '.join(words)
    ])

def save2customfile(pos_sent):
    '''
        保存到自定义文档中，主要用于增量训练反馈数据
    '''
    
    bert_sent = tag_sent_convter(pos_sent)
    if not bert_sent:
        return
    
    with open(custom_path, 'a') as f:
        f.write(bert_sent)
        f.write('\n')
    
    

pos_sents = [
    '周光/nr 明确/ad 否认/v CEO/ti 佟显侨/nr 和/c CTO/ti 衡量/nr 说/v 的/u 罪名/n ，/w 并/c 说会/v 跟/p 投资人/vn 、/w 技术leader/ti 一起/d 出来说明/v',
    '当时/t 其/r 2位/m 创始人/ti 佟显侨/nr 和/c 衡量/nr 推动发出/v 公司公告/n ，/w 把/p 还在/v 东京/ns 代表/v RoadStar/nt 出席/v 活动/n 的/u 另/r 一/m 联合创始人/ti 周光/nr 开除/v 。/w'
]

for pos_sent in pos_sents:
    save2customfile(pos_sent)


            